# demo for fully observed data

- in this example, we apply S3ID on fully observed data $y_t, t=1,\ldots,T$. See another demo for an application to spatiotemporally subsampled data
- we generate data from a linear dynamical system with $p=100$ observed and $n=4$ latent variables:
  $$ x_{t} = A x_{t-1} + \eta_t \\    
     y_t = C x_t + \epsilon_t, $$
     with emission noise $\epsilon_t \sim \mathcal{N}(0, R)$ for diagonal matrix $R \in \mathbb{R}_{+}^{p\times{}p}$ and innovation noise $\eta_t \sim \mathcal{N}(0, Q)$ with $Q\in\mathbb{R}^{n\times{}n}$
- we run S3ID to estimate the latent space $C\in\mathbb{R}^{p\times{}n}$, emission noise levels $R$ and time-lagged latent covariances $X_\tau = \mbox{cov}[x_{t+\tau},x_t] \in \mathbb{R}^{n\times{}n}$ 


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from S3ID import main, gen_data, ObservationScheme, principal_angle, comp_model_covariances

np.random.seed(42)


# define problem size
p, n, T = 100, 4, 1000   # number of observed variable, number of hidden variables, length of time-trace

# generate toy LDS parameters and data
pars_true, x, y, _, _ = gen_data(p,n,[],T)
y -= y.mean(axis=0) # ensure zero-mean input

plt.figure(figsize=(20,5))
plt.plot(x)
plt.xlabel('t')
plt.ylabel('x_t')
plt.title('latent dynamics (n = ' + str(n) + ')')
plt.show()

# compute first 2*n time-lagged empirical covariances
Qe = [np.cov(y[:T-k,:].T, y[k:,:].T)[p:,:p] for k in range(2*n)]

print('dynamics eigenvalues', np.linalg.eigvals(pars_true['A']))


# define observation scheme 

In [ ]:
obs_scheme = ObservationScheme(p,T) # simplest possible observation scheme: fully observed, 
                                    # i.e. observe all p variables for all T time points

# monitor subspace distance relatve to ground-truth parameters
- we track the **principal angles** between the column-spaces of ground-truth matrix $C$ and our current estimate of $C$ over epochs

In [ ]:
epochs = 150
proj_errors = np.zeros((epochs,n))
def pars_track(pars,t): 
    proj_errors[t] = principal_angle(pars_true['C'], pars[0])

# fit the model

In [ ]:
lag_range = np.arange(2*n) # matching time-lagged covariances for time-lags tau = 0, 1,.., 2*n-1  

pars_est, pars_init, traces, _, Om, W, t_desc = main(lag_range, n, y, 
                                                     obs_scheme, sso=True, 
                                                     parametrization='nl', # non-linear parametrization
                                                     batch_size=1, max_iter=epochs, 
                                                     pars_track=pars_track, verbose=True)
print('total time was ', t_desc)

# visualize results

In [ ]:
Qh = comp_model_covariances(pars_est, lag_range=lag_range) # model-predicted time-lagged covariances

plt.figure(figsize=(16,9))
plt.subplot(2,4,1)
plt.plot(traces[0])
plt.title('loss')
plt.xlabel('epochs')
plt.subplot(2,4,5)
plt.plot(proj_errors)
plt.title('principcal angles')
plt.xlabel('epochs')

plt.subplot(2,4,2)
plt.imshow(Qe[0], interpolation='None')
plt.title('empirical instant. cov.')
plt.subplot(2,4,3)
plt.imshow(Qh[0], interpolation='None')
plt.title('est. instant. cov.')
plt.subplot(2,4,4)
plt.plot(Qe[0], Qh[0], 'k+')
plt.xlabel('emp. inst. cov.')
plt.ylabel('est. inst. cov.')
plt.axis('square')

tau = 2*n-1
plt.subplot(2,4,6)
plt.imshow(Qe[tau], interpolation='None')
plt.title('empirical cov. at time-lag tau='+str(2*n-1))
plt.subplot(2,4,7)
plt.imshow(Qh[tau], interpolation='None')
plt.title('est. cov. at time-lag tau='+str(2*n-1))
plt.subplot(2,4,8)
plt.plot(Qe[tau], Qh[tau], 'k+')
plt.xlabel('emp. lagged cov.')
plt.ylabel('est. lagged cov.')
plt.axis('square')

plt.show()